# Practical 3
In this excercise, you will use what you have learnt from exercises 1 and 2 to build an SVM model to predict positive and negative classes from real biomedical data.

## Data origin
The data originates form the following publication:
https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1292-2
(going down to section Additional files - Additional file 3 will give you the full ist of raw data)
For the purpose of the exercise, we transformed the data already.
Before goint into downloading the dats - some common imports

In [9]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
from sklearn import svm # "Support vector classifier"
from sklearn import datasets
from mlxtend.plotting import plot_decision_regions
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [10]:
biomarkers_file_csv = 'clinical_biomarkers.csv'


df = pd.read_csv(biomarkers_file_csv)
df = df.set_index(['Sample'])

### Mapping classes into positive and negative
The following maps alle examples either to be positive or begative. Not matching ones ( 'C.'- Control ) are deleted

In [11]:
df_ex = df.copy()
df_ex['Response'] = df_ex['Response'].map(
    {
     'C.R.':'negative',
     'C.':'negative',
     'Int. II. R.':'negative',
     'High R.':'negative',
     'Int. I.':'positive',
     'Int. II.':'positive',
     'High':'positive',
    })


df_ex = df_ex.dropna()

In [12]:
#Let's look at the data
df_ex.head()

,Response,Hb (g/dL),RBC (mil/cmm),PCV (%),RET ABS(mil/cmm),MCV(fL),MCH (pg),MCHC (g/dL),RDW (%),PLAT (1000/cmm),...,IN PHOS(mmol/L),Cl (mmol/L),T PROT(g/L),ALBUMIN (g/L),TOT CHOL (mmol/L),GLUC (mmol/L),UREA (mmol/L),CREAT (umol/L),TRIGS (umol/L),CPK (IU/L)
Sample,,,,,,,,,,,,,,,,,,,,,
1,negative,0.092087,-0.933507,0.677989,1.571541,2.096461,1.518853,-1.198476,-0.248941,-2.229991,...,0.983719,-0.351491,-0.107674,-0.739030,1.107545,-2.501174,-0.185471,1.296293,-1.184046,1.092656
2,negative,0.255541,0.283637,-0.277325,-1.259227,-0.722841,-0.078755,1.080769,-1.174322,0.298847,...,0.058632,-0.351491,-0.847498,-0.984223,-0.784035,-1.150730,1.162481,1.862673,-1.363805,0.833558
3,negative,-0.234821,0.477273,-0.595763,-0.079740,-1.349353,-1.037321,0.776869,-0.526555,-0.778059,...,-0.403911,-1.044711,-0.600890,-1.229415,0.296868,-1.247191,2.095679,0.163532,-1.044234,-0.696923
4,negative,1.072810,1.058183,1.367938,-0.551535,0.269135,-0.238516,-0.742627,-1.544475,-0.210361,...,-0.866454,1.034947,-0.354282,-0.739030,1.647997,-0.186128,0.021906,-1.818798,-0.724662,-0.353469
5,negative,2.543896,1.611430,1.686377,-1.023329,-0.096330,0.720049,1.384668,-1.359399,-1.053307,...,-0.866454,0.341728,-0.107674,-0.493838,0.026642,-1.633032,1.473547,1.579483,-0.385117,-0.106423


Q1: Check how many data samples, and number of variables in the data set?

In [13]:
df_ex.shape
df_ex.Response.describe()

count           61
unique           2
top       positive
freq            32
Name: Response, dtype: object

In [14]:
df_ex = df_ex.values

In [15]:
#Prepare input and output
X = df_ex[:, 1:33].astype(float)
y = df_ex[:, 0]

#Change categorical data into numerical data
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
Q2: Separate data to train and test sets. Use 20% of data for testing. Set random state = 100

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [17]:
X_train.shape

(48, 32)

In [18]:
y_test

array([0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1])

Q3: Build a linear SVM model, using gamma = 'auto'. Let's call this model1.

In [19]:
model1 = svm.SVC(kernel = 'linear', gamma= 'auto')
model1.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Q4: Perform prediction and obtain the classification report

In [20]:
#Perform prediction
y_true, y_pred = y_test, model1.predict(X_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.50      0.50         6
           1       0.57      0.57      0.57         7

    accuracy                           0.54        13
   macro avg       0.54      0.54      0.54        13
weighted avg       0.54      0.54      0.54        13



Q5: Build a RBF SVM model, using gamma = 'auto. Let's call this model2. 
Perform prediction and obtain the classification report.

In [21]:
model2 = svm.SVC(kernel = 'rbf', gamma= 'auto')
model2.fit(X_train,y_train)
y_true, y_pred = y_test, model2.predict(X_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         6
           1       0.78      1.00      0.88         7

    accuracy                           0.85        13
   macro avg       0.89      0.83      0.84        13
weighted avg       0.88      0.85      0.84        13



Q6: Compare the results between model1 and model2.

Q7: Perform grid search to find the optimal C in linear SVM, and gamma and C in RBF SVM model. Use a grid search with 5-fold cross validation, score = 'f1', gamma = [1, 0.1, 0.01, 1e-3, 1e-4, 1e-5], and C = [0.1, 1, 10, 100, 1000]. 

In [22]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1, 0.1, 0.01, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}]

# Specify the evaluation metric we will use to select our hyper parameters. In this example, we will use auc.
scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5,
                       scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

Grid scores on development set:

0.685 (+/-0.046) for {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
0.768 (+/-0.136) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.782 (+/-0.121) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.695 (+/-0.080) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.685 (+/-0.046) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.685 (+/-0.046) for {'C': 1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.685 (+/-0.046) for {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
0.708 (+/-0.126) for {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.664 (+/-0.133) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.734 (+/-0.128) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.695 (+/-0.080) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.685 (+/-0.046) for {'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.685 (+/-0.046) for {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
0.708 (+/-0.126) for {'C': 100, '

/Users/laura/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Q8: Build the model with optimal hyperparameters. Let's call this model3. Perform prediction and obtain the classification report.

In [23]:
model3 = svm.SVC(kernel = 'rbf', gamma = 0.01, C = 1)
model3.fit(X_train, y_train)
y_true, y_pred = y_test, model3.predict(X_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         6
           1       0.70      1.00      0.82         7

    accuracy                           0.77        13
   macro avg       0.85      0.75      0.75        13
weighted avg       0.84      0.77      0.75        13



Q9: Compare the results between model1, model2, and model 3.